In [18]:
import requests as rq
import pymongo as pm
import ujson as js
import pandas as pd
from datetime import datetime as dt

from ApiHandler import ApiHandler

In [2]:
class ApiMarketOrders(ApiHandler):
    settings = {
        **ApiHandler.settings,
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/corporations/{corp_id}/orders/'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'Corp_MarketOrders',
    }
    name = 'ESI Market Orders API'
    params = {}
    auth = True
    
    def build_data(self, raw_data):
        data_frame = pd.DataFrame(js.loads(raw_data))
        data_frame['issued_corp'] = self.url_params['corp_id']
        data_frame['issued'] = data_frame['issued'].apply(lambda x: dt.strptime(x, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S'))
        data_frame.rename(columns={'range':'order_range'}, inplace=True)
        if 'is_buy_order' not in data_frame.columns: data_frame['is_buy_order'] = 0
        else: data_frame['is_buy_order'] = data_frame['is_buy_order'].fillna(0)
        data_frame['record_time'] = self.timestamp
        return data_frame

In [32]:
class ApiMarketOrdersHistory(ApiMarketOrders):
    settings = {
        **ApiMarketOrders.settings,
    }
    url = {
        **ApiMarketOrders.url,
        'data': 'https://esi.evetech.net/latest/corporations/{corp_id}/orders/history/'
    }
    sql = {
        **ApiMarketOrders.sql,
        'inactive_orders': """\
            SELECT CorpOrders.order_id,
                CorpOrders.record_time
            FROM Corp_MarketOrders AS CorpOrders
            WHERE CorpOrders.state != 'active'
                AND CorpOrders.order_id IN ({order_ids})
        ;"""
    }
    name = 'ESI Market Orders History API'
    
    def build_data(self, raw_data):
        data_frame = super().build_data(raw_data)
        inactive_orders = self.get_inactive_orders(data_frame['order_id'])
        data_frame.loc[
            data_frame['order_id'].isin(inactive_orders.index),
            'record_time'
        ] = data_frame.loc[
            data_frame['order_id'].isin(inactive_orders.index),
            'order_id'
        ].map(inactive_orders)
        return data_frame
        
    def get_inactive_orders(self, order_ids):
        self.connect_maria()
        sql_script = self.sql['inactive_orders'].format(order_ids=','.join(order_ids.astype(str)))
        inactive_orders = pd.read_sql(
            sql_script,
            self.conn['maria'],
            index_col='order_id'
        )['record_time'].apply(lambda x: x.to_pydatetime().isoformat(' ')).replace('NaT', None)
        self.conn['maria'].close()
        return inactive_orders

In [4]:
api = ApiMarketOrders()
api.run_process()

In [33]:
api = ApiMarketOrdersHistory()
api.run_process()